## Using the Home Assistant Python API
This notebook shows how you can work the [Home Assistant Python API](https://home-assistant.io/developers/python_api/).

### Setup

In [143]:
import homeassistant.remote as remote

In [146]:
HOST = '127.0.0.1'
PASSWORD = 'YOUR_PASSWORD'

api = remote.API(HOST, PASSWORD)

### Validation of the API is ready to use

In [147]:
print(remote.validate_api(api))

ok


### Retrieve the configuration
The Home Assistant Python API is not complete. Think of it as a sub-set of the [RESTful API](https://home-assistant.io/developers/rest_api/). This means that if a certain option is not available you can use Python's `requests` module to retrieve the data.

In [148]:
import requests

In [150]:
url = 'http://{}:8123/api/{}'.format(HOST, 'config')
headers = {'x-ha-access': PASSWORD,
           'content-type': 'application/json'}

config = requests.get(url, headers=headers).json()

(Starting with 0.25 `remote.get_config(api)` will be the short cut for this.)

### Display the current location

In [151]:
# This requires that geopy is installed, eg. pip3 install geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim()
location = geolocator.reverse("{}, {}".format(config.get('latitude'), config.get('longitude')))
location

Location(Bärengraben, 4, Grosser Muristalden, Gryphenhübeli, Bern, Verwaltungskreis Bern-Mittelland, Bern - Berne, 3006, Schweiz, Suisse, Svizzera, Svizra, (46.9480272, 7.45974119954229, 0.0))

In [152]:
factor = 0.001
box = ((config.get('latitude') - factor, config.get('longitude') - factor),
       (config.get('latitude') + factor, config.get('longitude') + factor))
marker = (config.get('latitude'), config.get('longitude'))

url = '{0}/export/embed.html?bbox={1}%2C{2}%2C{3}%2C{4}&amp;layer=mapnik&amp;marker={5}%2C{6}'.format(
            'https://www.openstreetmap.org', box[0][1], box[0][0], box[1][1], box[1][0], marker[0], marker[1])

In [153]:
iframe = '<iframe width="720" height="280" frameborder="0" scrolling="no" marginheight="0" marginwidth="0" src={} style="border: 1px solid black"></iframe>'.format(url)

In [154]:
from IPython.display import display, HTML
display(HTML(iframe))

There is also a so-called `magics` present for HTML. This will allow us to include a IFRAME as well. 

### Display the state of an entity

In [155]:
kitchen = remote.get_state(api, 'sensor.kitchen_temperature')

In [156]:
print('{} is {} {}.'.format(kitchen.attributes['friendly_name'],
                            kitchen.state,
                            kitchen.attributes['unit_of_measurement']
                            )
      )

Kitchen Temperature is unknown °C.
